In [1]:
import sys
import numpy as np
import tensorflow as tf
import os
from tensorflow.contrib.lookup import MutableHashTable
from tensor2tensor.layers import common_layers

from tensor2tensor.models import transformer
sys.path.append('/workspace/MT/tensor2tensor/tensor2tensor/models/')
sys.path.append('/workspace/MT/tensor2tensor/tensor2tensor/utils/')

from transformer_test import TransformerTest
from transformer_cache import TransformerCache
from tensor2tensor.data_generators import problem_hparams
from tensor2tensor.data_generators import problem
import beam_search_states

In [2]:
batch_size = 1
beam_size = 1
vocab_size = 2
decode_length = 3

initial_ids = tf.constant([0] * batch_size)  # GO
probabilities = tf.constant([[[0.7, 0.3]], [[0.4, 0.6]], [[0.5, 0.5]]])

expected_states = tf.constant([[[0.]], [[2.]]])

def symbols_to_logits(ids, _, states):
    pos = tf.shape(ids)[1] - 1
    # We have to assert the values of state inline here since we can't fetch
    # them out of the loop!
    with tf.control_dependencies(
      [tf.assert_equal(states["state"], expected_states[pos])]
    ):
        logits = tf.to_float(tf.log(probabilities[pos, :]))

    states["state"] += 2
    return logits, states

states = {
    "state": tf.zeros((batch_size, 1)),
}

states["state"] = tf.placeholder_with_default(
    states["state"], shape=(None, 1))

final_ids, _, st, fst = beam_search_states.beam_search(
    symbols_to_logits,
    initial_ids,
    beam_size,
    decode_length,
    vocab_size,
    0.0,
    eos_id=1,
    states=states
)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
  # Catch and fail so that the testing framework doesn't think it's an error
    try:
        f, s, ss = sess.run([final_ids, st, fst])
    except tf.errors.InvalidArgumentError as e:
        raise AssertionError(e.message)

In [10]:
batch_size = 1
beam_size = 2
vocab_size = 3
decode_length = 3

initial_ids = tf.constant([0] * batch_size)  # GO
probabilities = tf.constant([[[0.1, 0.1, 0.8], [0.1, 0.1, 0.8]],
                             [[0.4, 0.5, 0.1], [0.2, 0.4, 0.4]],
                             [[0.05, 0.9, 0.05], [0.4, 0.4, 0.2]]])

# The top beam is always selected so we should see the top beam's state
# at each position, which is the one thats getting 3 added to it each step.
expected_states = tf.constant([[[0.], [0.]], [[3.], [3.]], [[6.], [6.]]])

def symbols_to_logits(ids, _, states):
    pos = tf.shape(ids)[1] - 1

    # We have to assert the values of state inline here since we can't fetch
    # them out of the loop!
    with tf.control_dependencies(
      [tf.assert_equal(states["state"], expected_states[pos])]):
        logits = tf.to_float(tf.log(probabilities[pos, :]))

    states["state"] += tf.constant([[3.], [7.]])
    return logits, states

states = {
    "state": tf.zeros((batch_size, 1)),
}

states["state"] = tf.placeholder_with_default(
    states["state"], shape=(None, 1))

final_ids, _, st, fst = beam_search_states.beam_search(
    symbols_to_logits,
    initial_ids,
    beam_size,
    decode_length,
    vocab_size,
    0.0,
    eos_id=1,
    states=states)

with tf.Session() as sess:
  # Catch and fail so that the testing framework doesn't think it's an error
    sess.run(tf.global_variables_initializer())
    try:
        f, s, ss = sess.run([final_ids, st, fst])
    except tf.errors.InvalidArgumentError as e:
        raise AssertionError(e.message)

In [11]:
ss.shape

(1, 2, 4, 1)